In [ ]:
############################################## SETTINGS ##############################################  
import os
from os import walk
from tqdm import tqdm
import pickle
import random 
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
# Neuron
from   neuron           import h
import neuron as nrn
nrn.load_mechanisms('mechanisms/single')
# import custom functions
from   master_functions import *
#  Setup neuron
h.load_file('stdlib.hoc')
h.load_file('import3d.hoc')

############################################## SETTINGS ##############################################  
# original: original = True, method = 0 and physiological = False
# original = False        # inherited code
method = 0                # if 0 stimulate away from soma; if 1 then stimulate towards
physiological = True      # if False then uses original values else more physiological settings for phasic conductances 
# if model = 0 then original cell; if model = 1 then paper 
model = 1
cell_type='dspn'
specs = {'dspn': {
                    'N': 71,
                    'lib': 'Libraries/D1_71bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-dMSN_P270-20_1.02_SGA1-m24.swc',
                    },
         'ispn': {
                    'N': 34,
                    'lib': 'Libraries/D2_34bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-iMSN_P270-09_1.01_SGA2-m1.swc'}
        }


specs[cell_type]['model'] = model
morphology = specs[cell_type]['morph']
if model == 0:
    method = 0    
if model == 2:
    v_init = -84
else:
    v_init = -83.7155
return_currents = False
return_Ca = False
change_Ra = False
change_Cm = False
spine_per_length=1.711
change_spine_per_length = False

In [ ]:
sim = 12

In [ ]:
############################################## Figure 8 ############################################### 
######################################## Figure 6F, G and H ###########################################   
# glutamate submaximal
# glutamate timing 10 ms before
# does 3 sims: gaba only, glut only then gaba + glut
# onsite to stimulate 12 synapses 
# D, E, and F of figure 6
# Phasic glut
# model 1: 15 for subthreshold if gaba absent; 18 for standard
# model 1: 14 for subthreshold if gaba absent; 18? for standard
glut = True
if glut:
    if cell_type == 'dspn':
        num_gluts = 15
    elif cell_type == 'ispn':
        num_gluts = 12
else:
    num_gluts = 0

g_AMPA = 0.001 # original setting 0.001
AMPA = True
NMDA = True
ratio = 2.15 # NMDA to AMPA conductance ratio (not necessarily the current ratio)
glut_placement = 'distal'
if physiological:
    gaba_weight = 0.001
else:
    gaba_weight = 0.1 # tuned for non-distributed gaba (will be higher depolarization than ~5mv if running whole cell)

gaba_reversal = -60 # -83.7155 # 
# Phasic gaba
gaba = True
num_gabas = 3
if not gaba:
    num_gabas = 0

####
sim_time = 400
vary_gaba = False # if true timing of gaba is varied relative to glut; if false then vary glut relative to gaba

# if vary_gaba is True then gaba timing is given by timing  and glut timing is given by stim_time
# if vary_gaba is False then glut timing is given by timing and gaba timing is given by stim_time
stim_time = 150

spine_neck_diam = 0.1 # original setting 0.1
i_range = [3]

timing_range  = list(range(120, 201, 1))
timing_range.insert(0,150); timing_range.insert(0,150)

gaba_range = np.repeat(True, len(timing_range)-1)
gaba_range = np.insert(gaba_range, 1, False)
num_gabas_range = []
for x in gaba_range:
    num_gabas_range.append(x*num_gabas)
glut_range = np.repeat(True, len(timing_range)-1)
glut_range = np.insert(glut_range, 0, False)
num_gluts_range = []
for x in glut_range:
    num_gluts_range.append(x*num_gluts)
black_trace = 1
gray_trace = 0

impedance = False
# impedance is defined in frequency domain
freq = 10 

save = True

In [ ]:
# Save figures
def save_all():
    import datetime 
    time = datetime.datetime.now()

    path_cell = "{}".format(cell_type)
    if not os.path.exists(path_cell):
        os.mkdir(path_cell)    
    path1 = "{}/model{}".format(path_cell, model)
    if not os.path.exists(path1):
        os.mkdir(path1)
    if physiological: 
        path2 = "{}/physiological".format(path1)
    else:
        path2 = "{}/nonphysiological".format(path1)
    if not os.path.exists(path2):
        os.mkdir(path2)  
    path3 = "{}/images".format(path2)
    if not os.path.exists(path3):
        os.mkdir(path3)
    path4 = "{}/simulations".format(path2)
    if not os.path.exists(path4):
        os.mkdir(path4)  
    path5 = "{}/sim{}".format(path3, sim)
    if not os.path.exists(path5):
        os.mkdir(path5)    
    path6 = "{}/sim{}".format(path4, sim)
    if not os.path.exists(path6):
        os.mkdir(path6)  

    image_dir = "{}/{}".format(path5, protocol)
    if not os.path.exists(image_dir):
        os.mkdir(image_dir)  
    sim_dir = "{}/{}".format(path6, protocol)
    if not os.path.exists(sim_dir):
        os.mkdir(sim_dir)  

    fig_soma.write_image("{}/soma_{}.svg".format(image_dir, time))
    fig_dend.write_image("{}/dend_{}.svg".format(image_dir, time))
    if impedance:
        fig_z_soma.write_image("{}/z_soma_{}.svg".format(image_dir, time))
        fig_z_dend.write_image("{}/z_dend_{}.svg".format(image_dir, time))        
        
    with open('{}/{}_i_mechanisms.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(i_mechs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_v_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_v_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_vdend_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_vdend_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_Ca_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_Ca_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_Cadend_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_Cadend_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_t_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_t_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_gt.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(pickle_gt, handle, protocol=pickle.HIGHEST_PROTOCOL) 
    with open('{}/{}_AMPA.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(i_ampa, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_NMDA.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(i_nmda, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_gaba.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(i_gaba, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_g_gaba.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(g_gaba, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_v_tree.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(v_tree, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('{}/{}_Ca_tree.pickle'.format(sim_dir, protocol), 'wb') as handle:
        pickle.dump(Ca_tree, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    if impedance:
        with open('{}/{}_imp_dend_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
            pickle.dump(pickle_imp_dend_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('{}/{}_imp_soma_data.pickle'.format(sim_dir, protocol), 'wb') as handle:
            pickle.dump(pickle_imp_soma_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
 

In [ ]:
def gaba_and_glut(protocol, sim_time, plot, glut, glut_placement, branch1_glut, branch2_glut, branch3_glut, branch4_glut, branch5_glut, num_gluts, glut_onsets, gaba, gaba_placement, branch1_gaba, branch2_gaba, branch3_gaba, branch4_gaba, branch5_gaba, num_gabas, gaba_onsets, gaba_weight, spine_neck_diam, return_currents, cell_type='dspn', path=True):
    # create cell with spines
    cell, spines, dend_tree = cell_build(cell_type=cell_type, specs=specs, addSpines=True, spine_per_length=spine_per_length)
    if change_Ra:
        space_clamped(cell=cell, spines=spines, Ra=Ra)
        print(f"Ra is {Ra} ΜΩ")
    if change_Cm:
        cap(cell=cell, spines=spines, cm = Cm)
        print(f"Cm is {Cm} μFcm\u207B\u00B2")
        
    # change all spine neck diameters
    spine_neck_diameter(cell=cell, spines=spines, diam=spine_neck_diam)
                
    # Set up branch assignment and add glutamate
    glut_secs, glut_secs_orig = glut_add(
                    cell=cell,
                    branch1_glut = branch1_glut, 
                    branch2_glut = branch2_glut, 
                    branch3_glut = branch3_glut, 
                    branch4_glut = branch4_glut, 
                    branch5_glut = branch5_glut, 
                    num_gluts = num_gluts,
                    glut_placement = glut_placement,
                    glut = glut,
                    cell_type = cell_type)    
    
    # Place glutamatergic synapses
    glut_synapses, glut_stimulator, glut_connection, ampa_currents, nmda_currents = glut_place(
                    spines=spines,
                    method=method, 
                    physiological=physiological, 
                    AMPA=AMPA, 
                    g_AMPA=g_AMPA,
                    NMDA=NMDA,
                    ratio=ratio,
                    glut_time=glut_time,
                    glut_secs=glut_secs,
                    glut_onsets=glut_onsets,
                    num_gluts=num_gluts,
                    return_currents=return_currents)   
    
    # Set up branch assignment and add gaba
    gaba_secs = gaba_add(cell = cell,
                   gaba = gaba, 
                   branch1_gaba = branch1_gaba, 
                   branch2_gaba = branch2_gaba, 
                   branch3_gaba = branch3_gaba, 
                   branch4_gaba = branch4_gaba, 
                   branch5_gaba = branch5_gaba, 
                   gaba_placement = gaba_placement,
                   num_gabas = num_gabas,
                   cell_type = cell_type)
    
    # Place gabaergic synapses
    gaba_synapses, gaba_stimulator, gaba_connection, gaba_currents, gaba_conductances = gaba_place(
                   physiological = physiological,
                   gaba_reversal = gaba_reversal,
                   gaba_weight = gaba_weight,
                   gaba_time = gaba_time,
                   gaba_secs = gaba_secs,
                   gaba_onsets = gaba_onsets,
                   num_gabas = num_gabas,
                   return_currents = return_currents)
    
    if glut:
        dend = glut_secs[0] # first glut section is returned as v_dend
    elif gaba:
        dend = glut_secs_orig[0] # if no glut, show gaba at original 'glut site'
    else:
        dend = None    
    
    # Record vectors
    if path:
        pathlist = path_finder(cell=cell, dend_tree=dend_tree, dend=dend)
        v = record_v(cell=cell, seclist=pathlist, loc=0.4)
        ca = record_cai(cell, seclist=pathlist, loc=0.4, return_Ca=return_Ca)
    else:
        v = record_all_v(cell, loc=0.4)
        ca = record_all_cai(cell, seclist=pathlist, loc=0.4, return_Ca=return_Ca)
    t = h.Vector().record(h._ref_t)  
    i_mechs_dend = record_i_mechs(cell=cell, dend=dend, loc=0.4, return_currents=return_currents)

    # Initialize cell starting voltage
    h.finitialize(v_init)
    # Run simulation
    dt = 0.025
    h.dt = dt
    if impedance:
        # set up vectors
        imp, zvec1, zvec2 = record_impedance(dend=dend, loc=0.4)
        while h.t < sim_time:                
            imp.compute(freq,1)            
            zvec1.append(imp.input(0.4, sec=dend))    
            zvec2.append(imp.input(0.4, sec=cell.soma))    
            h.fadvance()
    else:
        while h.t < sim_time:
            h.fadvance()
    # gets secs to plot; if path is True, plots PSPs in each dendrite that forms part of dendritic path to soma
    if path:
        seclist = pathlist
    else:
        seclist = dend2plot(cell)
    
    tree_plots=False
    if tree_plots:
        fig_path = plot1(cell=cell, dend=dend, t=t, v=v, seclist=seclist, sparse=False, protocol='')
        fig_path.show()
        if return_Ca:
            fig_path_Ca = plot1_Ca(cell=cell, dend=dend, t=t, Ca=ca, seclist=seclist, sparse=False, protocol='')
            fig_path_Ca.show()
    
    if impedance:
        if return_Ca:
            return v['soma[0]'], v[dend.name()], v, ca['soma[0]'], ca[dend.name()], ca, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, zvec1, zvec2, dt
        else:
            return v['soma[0]'], v[dend.name()], v, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, zvec1, zvec2, dt            
    else:
        if return_Ca:
            return v['soma[0]'], v[dend.name()], v, ca['soma[0]'], ca[dend.name()], ca, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, dt
        else:
            return v['soma[0]'], v[dend.name()], v, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, dt


In [ ]:
soma_v_master = []
dend_v_master = []
if impedance:
    soma_z_master = []
    dend_z_master = []
for ii in tqdm(range(0, len(gaba_range), 1)): # gaba burst every 10 ms tqdm(range(140, 240, 10))
    soma_v_data = []
    dend_v_data = []
    soma_z_data = []
    dend_z_data = []
    v_data = []
    pickle_v_data = []
    pickle_vdend_data = []
    Ca_data = []
    pickle_Ca_data = []
    pickle_Cadend_data = []
    pickle_imp_dend_data = []
    pickle_imp_soma_data = []
    pickle_t_data = []
    pickle_gt = []
    i_mechs_dend_df = []

    v_tree = {}
    Ca_tree = {}
    i_mechs = {}
    i_ampa = {}
    i_nmda = {}
    i_gaba = {}
    g_gaba = {}

    gaba = gaba_range[ii]
    glut = glut_range[ii]
    num_gabas = num_gabas_range[ii]
    num_gluts = num_gluts_range[ii]
    timing = timing_range[ii]
    if change_Cm:
        Cm = Cm_range[ii]
    if change_spine_per_length:    
        spine_per_length = spine_per_length_range[ii]

    if vary_gaba:
        glut_time = stim_time
        gaba_time = timing
    else:
        glut_time = timing
        gaba_time = stim_time

    for i in i_range: # choose protocol to run # for i in [0, 1, 2, 3, 4]: #

        if i == 0: # on site distal
            if glut_placement == 'distal':
                proto = 'onsite'
            else:
                proto = 'onpath_dist'
            branch1_glut = False
            branch2_glut = True
            branch3_glut = False
            branch4_glut = False
            branch5_glut = False

            gaba_placement = 'distal_distributed_branch'
            branch1_gaba = False
            branch2_gaba = True
            branch3_gaba = False
            branch4_gaba = False
            branch5_gaba = False

        if i == 1: # on path proximal
            if glut_placement == 'distal':
                proto = 'onpath_prox'
            else:
                proto = 'onsite'
            branch1_glut = False
            branch2_glut = True
            branch3_glut = False
            branch4_glut = False
            branch5_glut = False

            gaba_placement = 'proximal_distributed_branch'
            branch1_gaba = False
            branch2_gaba = True
            branch3_gaba = False
            branch4_gaba = False
            branch5_gaba = False

        if i == 2: # off branch proximal
            proto = 'offpath_prox'
            branch1_glut = False
            branch2_glut = True
            branch3_glut = False
            branch4_glut = False
            branch5_glut = False

            gaba_placement = 'proximal_distributed_branch'
            branch1_gaba = True
            branch2_gaba = False
            branch3_gaba = True
            branch4_gaba = True
            branch5_gaba = True

        if i == 3: # off branch distal
            proto = 'offpath_dist'
            branch1_glut = False
            branch2_glut = True
            branch3_glut = False
            branch4_glut = False
            branch5_glut = False

            gaba_placement = 'distal_distributed_branch'
            branch1_gaba = True
            branch2_gaba = False
            branch3_gaba = True
            branch4_gaba = True
            branch5_gaba = True


        if i == 4: # somatic gaba
            proto = 'somatic'
            branch1_glut = False
            branch2_glut = True
            branch3_glut = False
            branch4_glut = False
            branch5_glut = False

            gaba_placement = 'soma'
            branch1_gaba = True
            branch2_gaba = False
            branch3_gaba = False
            branch4_gaba = False
            branch5_gaba = False

        # Phasic glut
        if model == 0:
            glut_onsets = list(range(glut_time, glut_time + num_gluts+1)) * (branch1_glut + branch2_glut + branch3_glut + branch4_glut + branch5_glut)
        else:
            glut_onsets = list(range(glut_time, glut_time + num_gluts)) * (branch1_glut + branch2_glut + branch3_glut + branch4_glut + branch5_glut)

        num_branch = branch1_glut + branch2_glut + branch3_glut + branch4_glut + branch5_glut
        num_branch2 = branch1_gaba + branch2_gaba + branch3_gaba + branch4_gaba + branch5_gaba

        # Burst gaba firing, 3 synapses per ms
        gaba_onsets = gaba_onset(gaba_time=gaba_time, num_gabas=num_gabas, num_branch2=num_branch2, model=model)

        if vary_gaba:
            protocol = 'GABA_' + format(num_branch) + 'nGLUT{}'.format(num_gluts) + '_' + format(num_branch2) + 'nGABA{}'.format(num_gabas) + '_GABA_timing{}'.format(gaba_time)
            protocol2 = 'GABA_' + proto + '_nGLUT{}'.format(num_gluts) + '_GABA_timing{}'.format(gaba_time)
        else:
            protocol = 'GLUT_' + format(num_branch) + 'nGLUT{}'.format(num_gluts) + '_' + format(num_branch2) + 'nGABA{}'.format(num_gabas) + '_GLUT_timing{}'.format(glut_time)
            protocol2 = 'GLUT_' + proto + '_nGLUT{}'.format(num_gluts) + '_GLUT_timing{}'.format(glut_time)

        if not i_range == [0,1,2,3,4]:
            sims = '_sim{}'.format( ''.join([str(i) for i in i_range]) ) # no gap
            # ' '.join([str(i) for i in i_range]) #  gives gap
            protocol = "{}{}".format(protocol, sims)

        if impedance:
            protocol = "{}{}".format(protocol, '_impedance')
        if change_Cm:
            protocol = "{}{}{}".format(protocol, '_Cm', Cm)
        if change_spine_per_length:
            protocol = "{}{}{}".format(protocol, '_spine_per_length', spine_per_length)
        # Run sim
        plot = True
        if impedance:
            if return_Ca:
                v_soma, v_dend, v_all, Ca_soma, Ca_dend, Ca_all, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, zvec1, zvec2, dt  = gaba_and_glut(protocol, sim_time, plot, glut, glut_placement, branch1_glut, branch2_glut, branch3_glut, branch4_glut, branch5_glut, num_gluts, glut_onsets, gaba, gaba_placement, branch1_gaba, branch2_gaba, branch3_gaba, branch4_gaba, branch5_gaba, num_gabas, gaba_onsets, gaba_weight, spine_neck_diam, return_currents, cell_type=cell_type)
            else:
                v_soma, v_dend, v_all, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, zvec1, zvec2, dt  = gaba_and_glut(protocol, sim_time, plot, glut, glut_placement, branch1_glut, branch2_glut, branch3_glut, branch4_glut, branch5_glut, num_gluts, glut_onsets, gaba, gaba_placement, branch1_gaba, branch2_gaba, branch3_gaba, branch4_gaba, branch5_gaba, num_gabas, gaba_onsets, gaba_weight, spine_neck_diam, return_currents, cell_type=cell_type)
        else:
            if return_Ca:
                v_soma, v_dend, v_all, Ca_soma, Ca_dend, Ca_all, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, dt = gaba_and_glut(protocol, sim_time, plot, glut, glut_placement, branch1_glut, branch2_glut, branch3_glut, branch4_glut, branch5_glut, num_gluts, glut_onsets, gaba, gaba_placement, branch1_gaba, branch2_gaba, branch3_gaba, branch4_gaba, branch5_gaba, num_gabas, gaba_onsets, gaba_weight, spine_neck_diam, return_currents, cell_type=cell_type)
            else:
                v_soma, v_dend, v_all, t, i_mechs_dend, ampa_currents, nmda_currents, gaba_currents, gaba_conductances, dt = gaba_and_glut(protocol, sim_time, plot, glut, glut_placement, branch1_glut, branch2_glut, branch3_glut, branch4_glut, branch5_glut, num_gluts, glut_onsets, gaba, gaba_placement, branch1_gaba, branch2_gaba, branch3_gaba, branch4_gaba, branch5_gaba, num_gabas, gaba_onsets, gaba_weight, spine_neck_diam, return_currents, cell_type=cell_type)
        if vary_gaba:
            v_tree['i:{} GABA time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)] = pd.DataFrame(v_all)
            if return_Ca:
                Ca_tree['i:{} GABA time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)] = pd.DataFrame(Ca_all)
        else:
            v_tree['i:{} GLUT time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)] = pd.DataFrame(v_all)
            if return_Ca:
                Ca_tree['i:{} GLUT time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)] = pd.DataFrame(Ca_all)

        ind1 = 1 
        ind2 = int(sim_time/dt)
        t2 = np.arange(1, int(sim_time/dt), 1) * dt - dt 
        soma_v_data.append(go.Scatter(x=t2, y=extract2(v_soma)[ind1:ind2], name='i:{}'.format(i)))
        dend_v_data.append(go.Scatter(x=t2, y=extract2(v_dend)[ind1:ind2], name='i:{}'.format(i)))
        soma_v_master.append(go.Scatter(x=t2, y=extract2(v_soma)[ind1:ind2], name='i:{} GLU:{} GABA:{}'.format(i, glut, gaba)))
        dend_v_master.append(go.Scatter(x=t2, y=extract2(v_dend)[ind1:ind2], name='i:{} GLU:{} GABA:{}'.format(i, glut, gaba)))
        if impedance:
            soma_z_data.append(go.Scatter(x=t2, y=extract2(zvec2)[ind1:ind2], name='i:{}'.format(i)))
            dend_z_data.append(go.Scatter(x=t2, y=extract2(zvec1)[ind1:ind2], name='i:{}'.format(i)))
            soma_z_master.append(go.Scatter(x=t2, y=extract2(zvec2)[ind1:ind2], name='i:{} GLU:{} GABA:{}'.format(i, glut, gaba)))
            dend_z_master.append(go.Scatter(x=t2, y=extract2(zvec1)[ind1:ind2], name='i:{} GLU:{} GABA:{}'.format(i, glut, gaba)))

        pickle_v_data.append(np.asarray(v_soma))
        if return_Ca:
            pickle_Ca_data.append(np.asarray(Ca_soma))
        pickle_t_data.append(np.asarray(t))
        pickle_gt.append([timing] * len(np.asarray(t)))
        pickle_vdend_data.append(np.asarray(v_dend))
        if return_Ca:
            pickle_Cadend_data.append(np.asarray(Ca_dend))
        if impedance:
            pickle_imp_dend_data.append(np.asarray(zvec1))
            pickle_imp_soma_data.append(np.asarray(zvec2))

        if return_currents:
            i_mechs_dend_df = list2df(i_mechs_dend)
            if vary_gaba:
                i_mechs['i:{} GABA time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)] = i_mechs_dend_df
                i_ampa['i:{} GABA time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)]  = pd.DataFrame(ampa_currents).transpose()
                i_nmda['i:{} GABA time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)]  = pd.DataFrame(nmda_currents).transpose()
                i_gaba['i:{} GABA time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)]  = pd.DataFrame(gaba_currents).transpose()
                g_gaba['i:{} GABA time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)]  = pd.DataFrame(gaba_conductances).transpose()
            else:
                i_mechs['i:{} GLUT time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)] = i_mechs_dend_df
                i_ampa['i:{} GLUT time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)]  = pd.DataFrame(ampa_currents).transpose()
                i_nmda['i:{} GLUT time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)]  = pd.DataFrame(nmda_currents).transpose()
                i_gaba['i:{} GLUT time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)]  = pd.DataFrame(gaba_currents).transpose()
                g_gaba['i:{} GLUT time:{} GABA:{} GLUT:{}'.format(i, proto, gaba, glut)]  = pd.DataFrame(gaba_conductances).transpose()

    fig_soma, fig_dend = plot2(soma_v_data, dend_v_data, glut_placement, yaxis = 'V (mV)') 
    if impedance:
        fig_z_soma, fig_z_dend = plot2(soma_z_data, dend_z_data, glut_placement, yaxis = 'impedance (MOhm)') 
    if save:
        save_all()

if model == 2 or cell_type == 'ispn':
    yrange_soma1=[-85,-50]
    yrange_dend1=[-85,-20]
else:
    yrange_soma1=[-85,-60]
    yrange_dend1=[-85,-30]  

if sim == 9 or sim == 10:
    fig_soma_master, fig_dend_master =  plot3(somaV=soma_v_master, dendV=dend_v_master, glut_placement=glut_placement, yaxis='V (mV)', yrange_soma=yrange_soma1, yrange_dend=yrange_soma1, stim_time=stim_time, sim_time=sim_time, black_trace=black_trace, gray_trace=gray_trace, err_bar=50, baseline=20, dt=dt)    
else:
    fig_soma_master, fig_dend_master =  plot3(somaV=soma_v_master, dendV=dend_v_master, glut_placement=glut_placement, yaxis='V (mV)', yrange_soma=yrange_soma1, yrange_dend=yrange_dend1, stim_time=stim_time, sim_time=sim_time, black_trace=black_trace, gray_trace=gray_trace, err_bar=50, baseline=20, dt=dt)    

fig_dend_master.show()
fig_soma_master.show()
if save: save_fig2(soma_fig=fig_soma_master, dend_fig=fig_dend_master, cell_type=cell_type, model=model, physiological=physiological, sim=sim)    
if impedance:
    if sim == 9 or sim == 10:
        fig_z_soma_master, fig_z_dend_master =  plot3(somaV=soma_z_master, dendV=dend_z_master, glut_placement=glut_placement, yaxis='MOhm', yrange_soma=[0,80], yrange_dend=[50,300], stim_time=stim_time, sim_time=sim_time, black_trace=black_trace, gray_trace=gray_trace, err_bar=50, baseline=20, dt=dt)    
    else:
        fig_z_soma_master, fig_z_dend_master =  plot3(somaV=soma_z_master, dendV=dend_z_master, glut_placement=glut_placement, yaxis='MOhm', yrange_soma=[0,100], yrange_dend=[0,800], stim_time=stim_time, sim_time=sim_time, black_trace=black_trace, gray_trace=gray_trace, err_bar=50, baseline=20, dt=dt)    
    fig_z_dend_master.show()
    fig_z_soma_master.show()
    if save: save_fig2(soma_fig=fig_z_soma_master, dend_fig=fig_z_dend_master, cell_type=cell_type, model=model, physiological=physiological, sim=sim)   
